# Exercises:

Answer the following questions.

## 1. Import the `Requests` package and call the [GitHub Jobs](https://jobs.github.com) API with the parameters of your choice. Print the status code to make sure it went well.

In [3]:
import requests
parameter={"full_time":True}
url="http://jobs.github.com/positions.json"
r=requests.get(url,params=parameter)
r.status_code

200

## 2. Import `json` and print the whole json response with the `json.dumps()` function. See what kind of data it contains.


In [4]:
import json
r.headers
data=r.json()
print (len(data))
print(json.dumps(data, sort_keys=True,indent=2))

50
[
  {
    "company": "GitHub",
    "company_logo": "http://github-jobs.s3.amazonaws.com/e54db6c2-080d-11e7-9d25-531751d26d63.png",
    "company_url": "https://github.com/about/jobs",
    "created_at": "Wed Apr 12 16:56:04 UTC 2017",
    "description": "<p>More and more companies are using GitHub Enterprise, so the Support team is growing. When our customers have problems, they contact us and it is our goal to help them as quickly and awesomely as possible. We&#39;re looking for an Enterprise Support Manager, in Asia Pacific, who will be responsible for the oversight and performance of a distributed staff answering customer support requests, triaging issues, and providing solutions. Candidates must currently be located in Australia, Japan, or New Zealand and possess legal working authorization.</p>\n\n<p>The position reports to the Director of Enterprise Support and works closely with other teams and leaders across the company.</p>\n\n<p>GitHub works remotely and asynchronously as mu

## 3. Get the dictionary of one of the jobs and call the `keys()` method to have a better idea of the data at hand.

In [3]:
data[0].keys()

dict_keys(['company', 'company_logo', 'created_at', 'id', 'location', 'how_to_apply', 'company_url', 'description', 'title', 'type', 'url'])

## 4. Print some of the first job's content, including images and HTML. You might need to import the `IPython` package.

In [4]:
from IPython.display import Image
image_url = data[0]['company_logo']
Image(url=image_url) 

In [5]:
from IPython.core.display import display, HTML
display(HTML(data[0]['description']))

In [6]:
print (data[0]['company'],
      data[0]['company_url'],
      data[0]['how_to_apply'],
    data[0]['created_at'], sep='\n')

GitHub
https://github.com/about/jobs
<p>Interested? We would love to hear more about you and your interest in joining us at GitHub! Just fill out our application <a href="https://boards.greenhouse.io/github/jobs/586192#.WMbOuxIrLBI">here</a>!</p>

Wed Apr 12 16:56:04 UTC 2017


## 5. Import `SQLite` and create a database.

In [5]:
import sqlite3
db=sqlite3.connect("database.db",detect_types=sqlite3.PARSE_DECLTYPES)
cursor=db.cursor()

## 6. Create 2 tables: `Company` and `Job`. Pay special attention on primary and foreign keys as well as unique and non-nullable columns. Make sure you store `created_at` as the equivalent of Python `datetime` and not `string`.


In [14]:
cursor.execute('CREATE TABLE IF NOT EXISTS company(company TEXT PRIMARY KEY,company_url TEXT,company_logo)')
db.commit()

In [7]:
cursor.execute('CREATE TABLE IF NOT EXISTS job(id TEXT PRIMARY KEY NOT NULL,title TEXT,description TEXT,company TEXT,how_to_apply TEXT, location TEXT, created_at TIMESTAMP, type TEXT,FOREIGN KEY (company) REFERENCES company(company))')
db.commit() 

In [10]:
#db.close()
#cursor.execute('DROP TABLE company')
#db.commit

<function Connection.commit>

In [6]:
#cursor.execute('DROP TABLE job')
#db.commit

<function Connection.commit>

## 7. Store the data you got from the API request in the database. Make sure you create a transaction (i.e., if an error occurs while saving a job's location data the database should rollback on all statements regarding that job).

In [16]:
for i in range(len(data)):
    comp=data[i]['company'] or ""
    url=data[i]['company_url'] or ""
    logo=data[i]['company_logo'] or ""
    cursor.execute("INSERT OR IGNORE INTO company(company,company_url,company_logo) VALUES ('"+comp+"','"+url+"','"+logo+"')")
db.commit()
# I got the follwing error: 'Can't convert 'NoneType' object to str implicitly', so I add an empty value. Is there another way to solve it?

In [8]:
import datetime
for i in range(len(data)):
    comp=data[i]['company'] or ""
    iid=data[i]['id'] or ""
    ti=data[i]['title'] or ""
    loc=data[i]['location'] or ""
    desc=data[i]['description'] or ""
    apply=data[i]['how_to_apply'] or ""
    cr=data[i]['created_at']
    ty=data[i]['type'] or ""             
    cursor.execute("INSERT OR IGNORE INTO job(id,title,description,company,how_to_apply,location,created_at,type) VALUES ('"+iid+"','"+ti+"','"+desc+"','"+comp+"','"+apply+"','"+loc+"','"+cr+"','"+ty+"')")
db.commit()

## 8. Execute simple SQL queries in each table and make sure everything looks as was intended.

In [9]:
cursor.execute('SELECT company, count(*) as count FROM company group by company')
rows=cursor.fetchall()

for row in rows:
    print(row)


('ASML', 1)
('AgriCircle', 1)
('Booking.com BV', 1)
('Booz Allen Hamilton', 1)
('Collide.com', 1)
('Contiamo', 1)
('FlashX', 1)
('GitHub', 1)
('Motive Interactive', 1)
('Nedap', 1)
('SugarCRM', 1)
('Torsh Inc', 1)
('Werk', 1)
('YNAB (You Need A Budget)', 1)


In [10]:
cursor.execute('SELECT company,max(created_at) last_job_from,count(*),max(location) FROM job group by company')
rows=cursor.fetchall()

for row in rows:
    print(row)

('ASML', 'Thu Mar 30 07:58:58 UTC 2017', 1, 'Veldhoven')
('AgriCircle', 'Fri Mar 31 13:46:51 UTC 2017', 1, 'Poland - Remote')
('Booking.com BV', 'Fri Mar 31 16:36:00 UTC 2017', 6, 'Amsterdam, the Netherlands')
('Booz Allen Hamilton', 'Thu Mar 30 22:30:23 UTC 2017', 31, 'Westborough, MA')
('Contiamo', 'Thu Mar 30 10:01:37 UTC 2017', 1, 'Berlin')
('European Stability Mechanism (ESM)', 'Mon Apr 03 12:52:31 UTC 2017', 1, 'Luxembourg')
('GitHub', 'Wed Mar 29 23:34:31 UTC 2017', 2, 'Remote- Australia')
('Motive Interactive', 'Sat Apr 01 02:49:04 UTC 2017', 1, 'San Diego, CA, US')
('Nedap', 'Fri Mar 31 07:45:21 UTC 2017', 1, 'Remote / Netherlands')
('Routehappy', 'Mon Apr 03 17:06:38 UTC 2017', 1, 'New York City')
('SugarCRM', 'Wed Mar 29 21:43:39 UTC 2017', 1, 'Cupertino, CA')
('Torsh Inc', 'Sun Apr 02 19:38:38 UTC 2017', 1, 'New Orleans')
('Werk', 'Sun Apr 02 19:33:05 UTC 2017', 1, 'New York')
('YNAB (You Need A Budget)', 'Thu Mar 30 20:40:01 UTC 2017', 1, 'Remote - Anywhere')


## 9. Create queries the following queries:
 * Find out how many jobs have the phrase `Data Science` and how many have the word `Python` in the description.
 * Find out how many were created in the past 7 days.
 * Find out how many jobs are located in `California` and which companies those are with.

**The output from our queries should be similar to this:**

There are 27 jobs with the 'Data Science' term, and 40% of them have 'Python' in the description.

22% of them have been created in the past 7 days.

2 of those are in California. The companies with those jobs are: ['Carbon Black, Inc.', 'Kaggle']

In [11]:
search='Data Science'
cursor.execute("SELECT company,location,description FROM job where description like '%{0}%'".format(search,))
rows=cursor.fetchall()
#with PYthon
search2='Python'
cursor.execute("SELECT count(*) FROM job where description like '%{0}%' and description like '%{1}%'".format(search,search2))
rows2=cursor.fetchone()

print("There are {0} jobs with the '{1}' term, and {2}% of them have '{3}' in the description.".format(len(rows) , search , round(rows2[0]/len(rows)*100,2) , search2))


There are 3 jobs with the 'Data Science' term, and 33.33% of them have 'Python' in the description.


In [14]:
#search='Data Science'
#cursor.execute("SELECT company,created_at FROM job where created_at-datetime('now')<7")
#rows=cursor.fetchall()
#it is not working!!!!!!!
cursor.execute("SELECT company,created_at FROM job ORDER by created_at asc")
rows=cursor.fetchall()

for row in rows:
    print(row)

ValueError: too many values to unpack (expected 2)

In [13]:
cursor.execute("SELECT distinct company FROM job where location like 'California' or location like '% CA%' ")
rows=cursor.fetchall()
company=[]
for row in rows:
    company.append(row[0])
print(len(rows), "of those are in California. The companies with those jobs are: ", company)


3 of those are in California. The companies with those jobs are:  ['Motive Interactive', 'Booz Allen Hamilton', 'SugarCRM']
